In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r'C:\Users\Jesse\Desktop'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## <center><b>Kaggle_Data_Science_Machine_Learning_Survey_2021</b></center>

# Importing the necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from plotly.offline import iplot
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
colors = ['#B1EDED','#B1B2ED','#1DE7ED','#1DA5ED','#1D50ED','#16548E']

## Importing the dataset

In [ ]:
survey_df = pd.read_csv('../input/kaggle-survey-2021-responses/kaggle_survey_2021_responses.csv')
survey_df.head()

In [ ]:
## Creating a copy of the original dataframe for data manipulation

survey_copy = survey_df.copy()
survey_copy.head()

# Exploring the Data

## Survey Duration in minutes

In [ ]:
survey_copy.rename(columns = {'Time from Start to Finish (seconds)-Duration (in seconds)' : 'Survey Time (Seconds)'} , inplace = True)
survey_copy['Survey Time (Seconds)'] = pd.to_numeric(survey_copy['Survey Time (Seconds)'])
# survey_mins = pd.DataFrame(survey_copy['Survey Time (Seconds)'])
survey_copy['Survey Time (Minutes)'] = survey_copy['Survey Time (Seconds)'] / 60
survey_copy['Survey Time (Minutes)'] = survey_copy['Survey Time (Minutes)'].astype('int64')

five_min_or_less = np.sum(survey_copy['Survey Time (Minutes)'] < 6)

six_to_ten_mins = np.sum((survey_copy['Survey Time (Minutes)'] > 5) & (survey_copy['Survey Time (Minutes)'] < 11))

eleven_to_twenty_mins = np.sum((survey_copy['Survey Time (Minutes)'] > 10) & (survey_copy['Survey Time (Minutes)'] < 21))

twentyone_to_thirty__mins = np.sum((survey_copy['Survey Time (Minutes)'] > 20) & (survey_copy['Survey Time (Minutes)'] < 31))

over_thirty_min = np.sum(survey_copy['Survey Time (Minutes)'] > 30)

values = [five_min_or_less , six_to_ten_mins , eleven_to_twenty_mins , twentyone_to_thirty__mins , over_thirty_min]

fig = go.Figure(go.Bar(y = values , marker_color = colors[2]))
fig.update_layout(template ="plotly_dark" , 
                  title = "Survey Duration" ,
                  xaxis_title = "Time (In Minutes)" ,
                  yaxis_title = "Number of People" ,
                  showlegend = False ,
                  xaxis = dict(
                  tickmode = 'array',
                  tickvals = [0,1, 2, 3, 4] ,
                  ticktext = ['< 5' , '6-10' , '11-20' , '21-30' , '30 +']) ,
                  font = dict(
                  family = "Courier New, monospace" ,
                  size = 18))

fig.update_xaxes(tickangle = 0)

<b>Observation</b>

- Majority of the participants tend to complete the survey between 6-20 minutes.

## Gender Distribution

In [ ]:
survey_copy['Q2-What is your gender? - Selected Choice'].value_counts()

## Replacing ['Prefer not to say , Nonbinary , Prefer to self-describe' to others]

survey_copy['Q2-What is your gender? - Selected Choice'].replace({'Prefer not to say':'Others' , 'Nonbinary':'Others' , 'Prefer to self-describe':'Others'} , inplace = True)

fig = go.Figure(go.Pie(labels = survey_copy['Q2-What is your gender? - Selected Choice'].value_counts().index , values = survey_copy['Q2-What is your gender? - Selected Choice'].value_counts().values , textinfo = 'label+percent'))
fig.update_traces(hole = .4 , marker = dict(colors = colors[2:]))
fig.update_layout(template="plotly_dark" , 
                  title_text="Gender Distribution",
                  # Add annotations in the center of the donut pies.
                  annotations=[dict(text='Gender', x=0.50, y=0.5, font_size=20, showarrow=False)])
fig.show()

<b>Observation</b>

- We notice that 80% of the distribution are Males and almost 20% of the distribution are Females.
- 2% of the distribution did not wish to disclose their gender.

## Age Distribution

In [ ]:
## Age Distribution

age_dist = survey_copy['Q1-What is your age (# years)?'].value_counts().reset_index()
age_dist.value_counts(ascending = False).reset_index()
age_dist.columns = ['Age Group' , 'Count']
age_dist.sort_values(by = 'Count' , ascending = False , inplace = True)
age_dist['percentage'] = round((age_dist['Count'] / age_dist['Count'].sum() * 100) , 2)
fig = go.Figure(go.Bar(x = age_dist['Age Group'] , y = age_dist['Count'] , text = age_dist['percentage'] , textposition='inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))

fig.update_layout(template="plotly_dark" , 
                  title = 'Age Distribution' , 
                  xaxis_title = 'Age (Years)' , yaxis_title = 'Number of participants')

<b>Observation</b>

- Majority of the participants fall within the age group of 18-29.
- We also notice that 0.50% of the participants are over the age of 70.

### Age Distribution by Gender

In [ ]:
man = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Man']['Q1-What is your age (# years)?'].value_counts()
woman = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Woman']['Q1-What is your age (# years)?'].value_counts()
others = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Others']['Q1-What is your age (# years)?'].value_counts() 

## Calculating the percentage for each group
pct_man = [round((m / (m + w + o)) * 100, 1) for m , w , o in zip(man.values , woman.values , others.values)]
pct_woman = [round((w / (m + w + o)) * 100, 1) for m , w , o in zip(man.values , woman.values , others.values)]
pct_others = [round((o / (m + w + o)) * 100, 1) for m , w , o in zip(man.values , woman.values , others.values)]

fig = go.Figure(data = [
                go.Bar(name = 'Man' , x = man.index , y = man.values , text = pct_man , marker_color = colors[2]) , 
                go.Bar(name = 'Woman' , x = woman.index , y = woman.values , text = pct_woman , marker_color = colors[3]) , 
                go.Bar(name = 'Others' , x = others.index , y = others.values , text = pct_others , marker_color = colors[4])])

fig.update_traces(texttemplate='%{text:.3s}%', textposition='inside')

fig.update_layout(template = 'plotly_dark' , 
                  barmode = 'stack' , 
                  title = 'Age Distribution By Gender' , 
                  xaxis_title = 'Age' , yaxis_title = 'Number of Participants')
fig.show()

<b>Observation</b>

- We notice that most of the Kaggle participants are between the age of 18 and 29.
- Majority of the participants are males.
- Based on this observation, we can say that most of the participants are just entering into the field of Data Science and Machine Learning.
- We also notice that there are some old participants who are still practicing Data Scientist as well.

## Country Distribution

In [ ]:
survey_copy['Q3-In which country do you currently reside?'] = survey_copy['Q3-In which country do you currently reside?'].replace({'United States of America' : 'USA' , 'United Kingdom of Great Britain and Northern Ireland' : 'UK' , 
                             'Viet Nam' : 'Vietnam' , 'Iran, Islamic Republic of...' : 'Iran'})
country_dist = survey_copy['Q3-In which country do you currently reside?'].value_counts().reset_index()
country_dist.value_counts(ascending = False).reset_index()
country_dist.columns = ['Country' , 'No. of Participants']
country_dist.sort_values(by = 'No. of Participants' , ascending = False , inplace = True)

fig = px.choropleth(data_frame = country_dist , locations = 'Country' , color = 'No. of Participants' , locationmode = 'country names' , 
                    hover_name = 'Country' , color_continuous_scale = colors ,  
                    title = 'Contry-Wise Distribution of Kaggle Participants')
fig.show()

<b>Observation</b>

- From the above geographic, we notice that India has the highest number of participants with a count of 7434, followed by USA with 2650 participants.
- The countries with the least number of participants are Iraq and Ethiopia with a total of 43 participants each.

### Gender-Wise Distribution of Kaggle Participants across Countries

In [ ]:
gender_by_country = pd.crosstab(index = survey_copy['Q3-In which country do you currently reside?'] , 
            columns = survey_copy['Q2-What is your gender? - Selected Choice'])
gender_by_country.reset_index(inplace = True)
gender_by_country.columns = ['Country' , 'Man' , 'Others' , 'Woman']

fig = px.choropleth(data_frame = gender_by_country , locations = 'Country' , locationmode = 'country names' , 
                    hover_name = 'Country' , hover_data = ['Man' , 'Others' , 'Woman'] ,   
                    title = 'Gender-Wise Distribution of Kaggle Participants across Countries')
fig.show()

<b>Observation</b>

- An observation that I would like to make is that India (1656) has the highest number of female participants followed by USA (606).
- The countries with the least amount of female participants are Peru (5) and Ethiopia (2)

### Country vs Age Distribution

In [ ]:
age_vs_ctry = pd.crosstab(index = survey_copy['Q3-In which country do you currently reside?'] , 
            columns = survey_copy['Q1-What is your age (# years)?'])
age_vs_ctry.columns = ["18-21" , "22-24" , "25-29" , "30-34" , "35-39" , "40-44" , "45-49" , "50-54" , "55-59" , "60-69" , "70+"]

plt.figure(figsize = (16,22));
sns.heatmap(age_vs_ctry.apply(lambda x: x/x.sum() , axis = 1) , annot = True , linewidths = 1 , cmap = "YlGn" , linecolor = 'black');
plt.xlabel('Age (Years)');
plt.ylabel('Country');
plt.xticks(rotation = 0);
plt.yticks(rotation = 0);
plt.title("Age Distribution based on the Country" , fontdict = {'fontsize' : 20});

<b>Observation</b>

- We notice that Vietnam contributes most the youngest participants (18-21 years).
- Bangladesh contributes most of the participants between 22 and 24 years.
- Uganda contributes most of the participants between 25 and 29 years.

## Level of Education

In [ ]:
survey_copy['Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].replace(
           {"Some college/university study without earning a bachelorâ€™s degree" : 'Graduand' , 
            'No formal education past high school' : 'High School' , 
            "Bachelorâ€™s degree" : "Bachelor's Degree" , 
            "Masterâ€™s degree" : "Master's Degree"} , inplace = True)

education_level = survey_copy['Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].value_counts().reset_index()
education_level.value_counts(ascending = False).reset_index()
education_level.columns = ['Level Of Education' , 'Count']
education_level.sort_values(by = 'Count' , ascending = False , inplace = True)
education_level['Percentage'] = round(education_level['Count'] / education_level['Count'].sum() *100 , 2)
fig = go.Figure(go.Bar(x = education_level['Level Of Education'] , y = education_level['Count'] , text = education_level['Percentage'] , textposition='inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))

fig.update_layout(template="plotly_dark" , 
                  title = 'Level of Education' , 
                  xaxis_title = 'Education Level' , yaxis_title = 'Number of participants')

<b>Observation</b>

- Majority of the population have completed their Bachelor's and Master's degree.

### Level of Education based on Gender

In [ ]:
man = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Man']['Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].value_counts()
woman = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Woman']['Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].value_counts()
others = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Others']['Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].value_counts() 

## Calculating the percentage for each group
pct_man = [round((m / (m + w + o)) * 100, 1) for m , w , o in zip(man.values , woman.values , others.values)]
pct_woman = [round((w / (m + w + o)) * 100, 1) for m , w , o in zip(man.values , woman.values , others.values)]
pct_others = [round((o / (m + w + o)) * 100, 1) for m , w , o in zip(man.values , woman.values , others.values)]

fig = go.Figure(data = [
                go.Bar(name = 'Man' , x = man.index , y = man.values , text = pct_man , marker_color = colors[2]) , 
                go.Bar(name = 'Woman' , x = woman.index , y = woman.values , text = pct_woman , marker_color = colors[3]) , 
                go.Bar(name = 'Others' , x = others.index , y = others.values , text = pct_others , marker_color = colors[4])])

fig.update_traces(texttemplate='%{text:.3s}%', textposition='inside')

fig.update_layout(template = 'plotly_dark' , 
                  barmode = 'stack' , 
                  title = 'Level of Education based on Gender' , 
                  xaxis_title = 'Level of Education' , yaxis_title = 'Number of Participants')
fig.show()

<b>Observation</b>

- We observe that majority of the participants have either a Bachelor's or a Master's degree.
- Also, majority of the participants are males in each degree given that the population is male dominant.

### Level of Education based on Age

In [ ]:
edu_vs_age = pd.crosstab(index = survey_copy['Q1-What is your age (# years)?'] , 
            columns = survey_copy['Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'])
edu_vs_age.columns = ["Bachelor's Degree" , "Doctoral degree" , "Graduand" , "High School" , "I prefer not to answer" ," Master's Degree" , "Professional doctorate"]

plt.figure(figsize = (14,10));
sns.heatmap(edu_vs_age.apply(lambda x: x/x.sum() , axis = 1) , annot = True , linewidths = 1 , cmap = "YlGn" , linecolor = 'black');
plt.xlabel('Level of Education');
plt.ylabel('Age (Years)');
plt.xticks(rotation = 90);
plt.yticks(rotation = 0);
plt.title("Planned / Current Education Distribution by Age" , fontdict = {'fontsize' : 20});

<b>Observation</b>

- We clearly notice that 68% of the participants who either completed or planning to do a Bachelor's Degree are within within the age range of 18-21, closely followed by 22-24.
- This tells us that they are students fresh out of high school within the age range of 18-24.
- We also notice that the next highest distribution are participants who either did or want to do a Master's Degree.
- There is a wide age range for people who want to complete their Master's Degree, majority ranging from 25-34.
- It is safe to say that as the participants get older, they want to complete higher degrees.

### Level of Education based on Country

In [ ]:
edu_vs_ctry = pd.crosstab(index = survey_copy['Q3-In which country do you currently reside?'] , 
            columns = survey_copy['Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'])
edu_vs_ctry.columns = ["Bachelor's Degree" , "Doctoral degree" , "Graduand" , "High School" , "I prefer not to answer" ," Master's Degree" , "Professional doctorate"]

plt.figure(figsize = (16,22));
sns.heatmap(edu_vs_ctry.apply(lambda x: x/x.sum() , axis = 1) , annot = True , linewidths = 1 , cmap = "YlGn" , linecolor = 'black');
plt.xlabel('Level of Education');
plt.ylabel('Country');
plt.xticks(rotation = 90);
plt.yticks(rotation = 0);
plt.title("Planned / Current Education Distribution by Country" , fontdict = {'fontsize' : 20});

<b>Observation</b>

- We notice that Kenya has the highest number of Kaggle participants who have completed their Bachelor's Degree(66%), closely followed by SriLanka and Egypt (both with 61%).
- We notice that France has the highest number of Kaggle participants who have completed their Master's Degree(70%) , closely followed by Belgium , Czech Republic and Sweden (all with 63%).
- From this we notice that European Countries have a higher number of participants who have completed their Master's Degree.

### Country-Age-Education Distribution

In [ ]:
fig = px.treemap(survey_copy , path = [px.Constant('Countries') , 
                                 'Q3-In which country do you currently reside?' , 'Q1-What is your age (# years)?' , 
                                 'Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'])

fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(title = 'Country-Age-Education Distribution')

<b>Observation</b>

- The above treemap shows the participants degree level according to which country they are from and what their age group is.

## Job Role of the participants

In [ ]:
survey_copy['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'] = survey_copy['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].replace({'Currently not employed' : 'Currently Unemployed'})
survey_copy['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts()

designation = survey_copy['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts().reset_index()
designation.columns = ['Designation' , 'Count']
designation.sort_values(by = 'Count' , ascending = True , inplace = True)
designation['Percentage'] = round(designation['Count'] / designation['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = designation['Count'] , y = designation['Designation'] , orientation = 'h' , text = designation['Percentage'] , textposition='inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))

fig.update_layout(template = 'plotly_dark' , 
                  title = 'Current / Most Recent Role of the Participants' , 
                  xaxis_title = 'Number of Participants' , 
                  yaxis_title = 'Current / Most Recent Role')

<b>Observation</b>

- As we can see, majority of the participants are Students, followed by Data Scientists.
- It is glad to see how students are showing an interest in the future of data science / machine learning given that the future is going to revlove around data.

### Job Role based on Gender

In [ ]:
job_man = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Man']['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts()
job_woman = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Woman']['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts()
job_others = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Others']['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts() 

## Calculating the percentage for each group
pct_man = [round((m / (m + w + o)) * 100, 1) for m , w , o in zip(job_man.values , job_woman.values , job_others.values)]
pct_woman = [round((w / (m + w + o)) * 100, 1) for m , w , o in zip(job_man.values , job_woman.values , job_others.values)]
pct_others = [round((o / (m + w + o)) * 100, 1) for m , w , o in zip(job_man.values , job_woman.values , job_others.values)]

fig = go.Figure(data = [
                go.Bar(name = 'Man' , x = job_man.index , y = job_man.values , text = pct_man , marker_color = colors[2]) , 
                go.Bar(name = 'Woman' , x = job_woman.index , y = job_woman.values , text = pct_woman , marker_color = colors[3]) , 
                go.Bar(name = 'Others' , x = job_others.index , y = job_others.values , text = pct_others , marker_color = colors[4])])

fig.update_traces(texttemplate='%{text:.3s}%', textposition='inside')

fig.update_layout(template = 'plotly_dark' , 
                  barmode = 'stack' , 
                  title = 'Job Role based on Gender' , 
                  xaxis_title = 'Job Role' , yaxis_title = 'Number of Participants')

### Job Role based on Age

In [ ]:
role_vs_age = pd.crosstab(index = survey_copy['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'] , 
            columns = survey_copy['Q1-What is your age (# years)?'])
role_vs_age.columns = ["18-21" , "22-24" , "25-29" , "30-34" , "35-39" , "40-44" , "45-49" , "50-54" , "55-59" , "60-69" , "70+"]

plt.figure(figsize = (14,10));
sns.heatmap(role_vs_age.apply(lambda x: x/x.sum() , axis = 1) , annot = True , linewidths = 1 , cmap = "YlGn" , linecolor = 'black');
plt.xlabel('Age (Years)');
plt.ylabel('Level of Education');
plt.xticks(rotation = 90);
plt.yticks(rotation = 0);
plt.title("Job Role by Age" , fontdict = {'fontsize' : 20});

<b>Observation</b>

- We notice that almost 80% of the participants within the age group of 18-24 are Students.
- We also notice that participants get older, they get into better Job positions (eg. Product/Program Manager)
- The highest number of participants who are either Data Scientists or Machine Learning Engineers are within the age group of 22-34.

### Job Role based on the Country

In [ ]:
role_vs_ctry = pd.crosstab(index = survey_copy['Q3-In which country do you currently reside?'] , 
            columns = survey_copy['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'])
role_vs_ctry.columns = ["Business Analyst" , "Currently Unemployed" , "DBA/Database Engineer" , "Data Analyst" , "Data Engineer" , 
                        "Data Scientist" , "Developer Relations/Advocacy" , "Machine Learning Engineer" , "Other" , 
                        "Product Manager" , "Program/Project Manager" , "Research Scientist" , "Software Engineer" , "Statistician" , "Student"]

plt.figure(figsize = (16,22));
sns.heatmap(role_vs_ctry.apply(lambda x: x/x.sum() , axis = 1) , annot = True , linewidths = 1 , cmap = "YlGn" , linecolor = 'black');
plt.xlabel('Job Role');
plt.ylabel('Country');
plt.xticks(rotation = 90);
plt.yticks(rotation = 0);
plt.title("Job Role based on the Country" , fontdict = {'fontsize' : 20});

<b>Observation</b>

- We notice that majority of the Students who are participants are from China and Bangladesh, closely followed by Sri Lanka.
- Most of the Machine Learning Engineers are from Iraq, and most are the Data Scientists are from France and Denmark closely followed by Israel.
- 20% of the participants who chose 'Other' as an option are from United Arab Emirates.
- 26% of the participants who are currently unemployed are from Uganda.

### Job Role based on Education Level

In [ ]:
fig = px.sunburst(survey_copy , path = ['Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?' , 
                                 'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'])

fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Job Role based on Education' , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above plot breaks up the role of participants based on their education level.

### Country-Age-Education-Role Distribution

In [ ]:
fig = px.treemap(survey_copy , path = [px.Constant('Countries') , 
                                 'Q3-In which country do you currently reside?' , 'Q1-What is your age (# years)?' , 
                                 'Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?' , 
                                 'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'])

fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(title = 'Country-Age-Education-Role Distribution')

<b>Observation</b>

- The above treemap breaks up the role of participants based which country they are from, their age group, education level and their current/recent job role.

## Coding Experience

In [ ]:
code_exp = survey_copy['Q6-For how many years have you been writing code and/or programming?'].value_counts().reset_index()
code_exp.columns = ['Coding Experience' , 'Count']
code_exp['Coding Experience'].replace({'I have never written code' : 'No coding experience'} , inplace = True)
code_exp['Percentage'] = round(code_exp['Count'] / code_exp['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = code_exp['Coding Experience'] , y = code_exp['Count'] , text = code_exp['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Coding Experience' , 
                  xaxis = dict(tickmode = 'array' , tickvals = [0,1,2,3,4,5,6] , ticktext = ['1-3','< 1','3-5','5-10','10-20','20 +' , 'No experience']) , 
                  xaxis_title = 'Coding Experience (Years)' , yaxis_title = 'Number of participants')

<b>Observation</b>

- Majority of the participants have 1-3 years of coding experience, given that most of the participants are students who are of young age.

### Coding Experience by Gender

In [ ]:
code_man = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Man']['Q6-For how many years have you been writing code and/or programming?'].value_counts()
code_woman = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Woman']['Q6-For how many years have you been writing code and/or programming?'].value_counts()
code_others = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Others']['Q6-For how many years have you been writing code and/or programming?'].value_counts() 

## Calculating the percentage for each group
pct_man = [round((m / (m + w + o)) * 100, 1) for m , w , o in zip(code_man.values , code_woman.values , code_others.values)]
pct_woman = [round((w / (m + w + o)) * 100, 1) for m , w , o in zip(code_man.values , code_woman.values , code_others.values)]
pct_others = [round((o / (m + w + o)) * 100, 1) for m , w , o in zip(code_man.values , code_woman.values , code_others.values)]

fig = go.Figure(data = [
                go.Bar(name = 'Man' , x = code_man.index , y = code_man.values , text = pct_man , marker_color = colors[2]) , 
                go.Bar(name = 'Woman' , x = code_woman.index , y = code_woman.values , text = pct_woman , marker_color = colors[3]) , 
                go.Bar(name = 'Others' , x = code_others.index , y = code_others.values , text = pct_others , marker_color = colors[4])])

fig.update_traces(texttemplate='%{text:.3s}%', textposition='inside')

fig.update_layout(template = 'plotly_dark' , 
                  barmode = 'stack' , 
                  title = 'Coding Experience by Gender' , 
                  xaxis_title = 'Coding Experience' , yaxis_title = 'Number of Participants')

### Coding Experience based on Job Role

In [ ]:
codexp_vs_role = pd.crosstab(index = survey_copy['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice'] , 
            columns = survey_copy['Q6-For how many years have you been writing code and/or programming?'])
codexp_vs_role.columns = ["1-3 years" , "10-20 years" , "20+ years" , "3-5 years" , "5-10 years" , "< 1 years" , "I have never written code"]

plt.figure(figsize = (14,10));
sns.heatmap(codexp_vs_role.apply(lambda x: x/x.sum() , axis = 1) , annot = True , linewidths = 1 , cmap = "YlGn" , linecolor = 'black');
plt.xlabel('Coding Experience');
plt.ylabel('Job Role');
plt.xticks(rotation = -60);
plt.yticks(rotation = 0);
plt.title("Coding Experience based on Job Role" , fontdict = {'fontsize' : 20});

<b>Observation</b>

- For coding experience less than a year, majority of the participants are currently umemployed, followed by the participants who are students.
- Majority of the participants who have coding experience somewhere between 1-3 years are again students, followed by Data Analysts and participants who are unemployed.
- Most of the participants who have 20+ years of coding experience are either Program Manager or Research Scientist or they are a DataBase Engineer/DBA.
- Most of the participants who have no coding experience at all are either Business Analysts or Product Manager or Others.

### Coding Experience by each Country

In [ ]:
code_vs_ctry = pd.crosstab(index = survey_copy['Q3-In which country do you currently reside?'] , 
            columns = survey_copy['Q6-For how many years have you been writing code and/or programming?'])
code_vs_ctry.columns = ["1-3 years" , "10-20 years" , "20+ years" , "3-5 years" , "5-10 years" , "< 1 years" , "I have never written code"]

plt.figure(figsize = (16,22));
sns.heatmap(code_vs_ctry.apply(lambda x: x/x.sum() , axis = 1) , annot = True , linewidths = 1 , cmap = "YlGn" , linecolor = 'black');
plt.xlabel('Coding Experience');
plt.ylabel('Country');
plt.xticks(rotation = -60);
plt.yticks(rotation = 0);
plt.title("Coding Experience by each Country" , fontdict = {'fontsize' : 20});

<b>Observation</b>

- Half the participants who have 1-3 years of coding experience are from Tunisia, followed by Nepal and China.
- Most of the participants with less than 1 year experience are from Uganda, followed by Kazakhstan and Kenya.
- Most of the participants who have never coded before are from the United Arab Emirates.

### Coding Experience based on Education Level

In [ ]:
fig = px.sunburst(survey_copy , path = ['Q4-What is the highest level of formal education that you have attained or plan to attain within the next 2 years?' , 
                                        'Q6-For how many years have you been writing code and/or programming?'])

fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Coding Experience based on Education' , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above plot breaks up the role of the participants knowledge of coding based on their education level.

## Programming Languages

In [ ]:
prog_lang = [col for col in survey_copy if col.startswith('Q7')]
prog_lang = survey_copy[prog_lang]
prog_lang.columns = ['Python' , 'R' , 'SQL' , 'C' , 'C++' , 'Java' , 'JavaScript' , 'Julia' , 'Swift' , 'Bash' , 
                     'Matlab' , 'None' , 'Other']
prog_lang = (prog_lang.count().to_frame().reset_index().rename(columns = {'index' : 'Language' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
prog_lang['Percentage'] = round(prog_lang['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = prog_lang['Count'] , y = prog_lang['Language'] , orientation = 'h' , text = prog_lang['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Most Commonly used Programming Languages' , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'Language')

<b>Observation</b>

- We see that Python is the most commonly used programming language, given that it is simple and easy to understand and implement.
- Python and R seem to be the languages of the modern day world. Most of the data scientists either use Python or R for data analysis.
- SQL is second on the list of the commonly used programming languages given that SQL is evergreen and it is widely used in any sort of data mining or data analysis.
- Julia and Swift take the bottom two places of the commonly used languages.

### Programming language split by gender

In [ ]:
id_vars = ['Q2-What is your gender? - Selected Choice']
value_vars = ['Q7_Part_1-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python',
 'Q7_Part_2-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R',
 'Q7_Part_3-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL',
 'Q7_Part_4-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C',
 'Q7_Part_5-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C++',
 'Q7_Part_6-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Java',
 'Q7_Part_7-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Javascript',
 'Q7_Part_8-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Julia',
 'Q7_Part_9-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Swift',
 'Q7_Part_10-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Bash',
 'Q7_Part_11-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - MATLAB',
 'Q7_Part_12-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - None',
 'Q7_OTHER-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Other']

prog_by_gender = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
prog_by_gender.dropna(inplace = True)
fig = px.sunburst(prog_by_gender , path = ['Q2-What is your gender? - Selected Choice' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Commonly used programming language by gender' , 
                  height = 700 , width = 1000)

### Programming language based on Job Role

In [ ]:
id_vars = ['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice']
value_vars = ['Q7_Part_1-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python',
 'Q7_Part_2-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R',
 'Q7_Part_3-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL',
 'Q7_Part_4-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C',
 'Q7_Part_5-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C++',
 'Q7_Part_6-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Java',
 'Q7_Part_7-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Javascript',
 'Q7_Part_8-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Julia',
 'Q7_Part_9-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Swift',
 'Q7_Part_10-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Bash',
 'Q7_Part_11-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - MATLAB',
 'Q7_Part_12-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - None',
 'Q7_OTHER-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Other']

prog_by_role = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
prog_by_role.dropna(inplace = True)
fig = px.sunburst(prog_by_role , path = ['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Commonly used programming language based on the job role' , 
                  height = 700 , width = 1000)

### Programming language distributed by age

In [ ]:
id_vars = ['Q1-What is your age (# years)?']
value_vars = ['Q7_Part_1-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python',
 'Q7_Part_2-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R',
 'Q7_Part_3-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL',
 'Q7_Part_4-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C',
 'Q7_Part_5-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C++',
 'Q7_Part_6-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Java',
 'Q7_Part_7-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Javascript',
 'Q7_Part_8-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Julia',
 'Q7_Part_9-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Swift',
 'Q7_Part_10-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Bash',
 'Q7_Part_11-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - MATLAB',
 'Q7_Part_12-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - None',
 'Q7_OTHER-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Other']

prog_by_age = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
prog_by_age.dropna(inplace = True)
fig = px.treemap(prog_by_age , path = [px.Constant('Age') , 'Q1-What is your age (# years)?' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Commonly used programming language based on age' , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above treemap gives a breakdown of the commonly used languages based on the age group of the participants.
- Clearly, we notice across all age groups Python unanimously takes the majority of votes. That is how much Python is in demand.

### Programming language split by countries

In [ ]:
id_vars = ['Q3-In which country do you currently reside?']
value_vars = ['Q7_Part_1-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python',
 'Q7_Part_2-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R',
 'Q7_Part_3-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL',
 'Q7_Part_4-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C',
 'Q7_Part_5-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C++',
 'Q7_Part_6-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Java',
 'Q7_Part_7-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Javascript',
 'Q7_Part_8-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Julia',
 'Q7_Part_9-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Swift',
 'Q7_Part_10-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Bash',
 'Q7_Part_11-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - MATLAB',
 'Q7_Part_12-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - None',
 'Q7_OTHER-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Other']

prog_by_country = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
prog_by_country.dropna(inplace = True)
fig = px.treemap(prog_by_country , path = [px.Constant('Countries') , 
                                           'Q3-In which country do you currently reside?' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Commonly used programming language based on country' , 
                  height = 700 , width = 1000)

### Commonly used programming language based on programming experience

In [ ]:
id_vars = ['Q6-For how many years have you been writing code and/or programming?']
value_vars = ['Q7_Part_1-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python',
 'Q7_Part_2-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R',
 'Q7_Part_3-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL',
 'Q7_Part_4-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C',
 'Q7_Part_5-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C++',
 'Q7_Part_6-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Java',
 'Q7_Part_7-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Javascript',
 'Q7_Part_8-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Julia',
 'Q7_Part_9-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Swift',
 'Q7_Part_10-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Bash',
 'Q7_Part_11-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - MATLAB',
 'Q7_Part_12-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - None',
 'Q7_OTHER-What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Other']

prog_by_country = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
prog_by_country.dropna(inplace = True)
fig = px.sunburst(prog_by_country , path = ['Q6-For how many years have you been writing code and/or programming?' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Commonly used programming language based on programming experience' , 
                  height = 700 , width = 1000)

# Programming language recommendation

In [ ]:
lang_recommend = survey_copy['Q8-What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice'].value_counts().reset_index()
lang_recommend.columns = ['Language' , 'Count']
lang_recommend.sort_values(by = 'Count' , ascending = True , inplace = True)
lang_recommend['Percentage'] = round(lang_recommend['Count'] / lang_recommend['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = lang_recommend['Count'] , y = lang_recommend['Language'] , orientation = 'h' , text = lang_recommend['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Most recommended programming language' , 
                 xaxis_title = 'Number of recommendations' , yaxis_title = 'Language')

<b>Observation</b>

- Given that Python is the most widely used programming language, it is not a surprise that the most recommended language is Python.

# Most commonly used IDE's

In [ ]:
ide = [col for col in survey_copy if col.startswith('Q9')]
ide = survey_copy[ide]
ide.columns = ['Jupyter Lab' , 'RStudio' , 'Visual Studio' , 'VSCode' , 'PyCharm' , 'Spyder' , 'Notepad++' , 
               'Sublime Text' , 'Vim / Emacs' , 'MATLAB' , 'Jupyter Notebook' , 'None' , 'Other']
ide = (ide.count().to_frame().reset_index().rename(columns = {'index' : 'IDE' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
ide['Percentage'] = round(ide['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = ide['Count'] , y = ide['IDE'] , orientation = 'h' , text = ide['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most Commonly used IDE's" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'IDE')

<b>Observation</b>

- Given that Python is the most widely used programming language, Jupyter Notebook seems to be the choice for an IDE.
- The reason maybe due to the interpretability. It interprets each row one at a time and does not compile the whole code like other languages do.
- Jupyter Notebook is the goto choice for Data Scientists and Machine Learners.

## Commonly used IDE's based on gender

In [ ]:
id_vars = ['Q2-What is your gender? - Selected Choice']
value_vars = ["Q9_Part_1-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - Jupyter (JupyterLab, Jupyter Notebooks, etc) ",
 "Q9_Part_2-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  RStudio ",
 "Q9_Part_3-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Visual Studio ",
 "Q9_Part_4-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Visual Studio Code (VSCode) ",
 "Q9_Part_5-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  PyCharm ",
 "Q9_Part_6-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Spyder  ",
 "Q9_Part_7-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Notepad++  ",
 "Q9_Part_8-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Sublime Text  ",
 "Q9_Part_9-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Vim / Emacs  ",
 "Q9_Part_10-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  MATLAB ",
 "Q9_Part_11-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Jupyter Notebook",
 "Q9_Part_12-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - None",
 "Q9_OTHER-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - Other"]

ide_by_gender = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
ide_by_gender.dropna(inplace = True)
fig = px.sunburst(ide_by_gender , path = ['Q2-What is your gender? - Selected Choice' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used IDE's based on gender" , 
                  height = 700 , width = 1000)

## Commonly used IDE's based on Job Role

In [ ]:
id_vars = ['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice']
value_vars = ["Q9_Part_1-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - Jupyter (JupyterLab, Jupyter Notebooks, etc) ",
 "Q9_Part_2-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  RStudio ",
 "Q9_Part_3-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Visual Studio ",
 "Q9_Part_4-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Visual Studio Code (VSCode) ",
 "Q9_Part_5-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  PyCharm ",
 "Q9_Part_6-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Spyder  ",
 "Q9_Part_7-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Notepad++  ",
 "Q9_Part_8-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Sublime Text  ",
 "Q9_Part_9-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Vim / Emacs  ",
 "Q9_Part_10-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  MATLAB ",
 "Q9_Part_11-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Jupyter Notebook",
 "Q9_Part_12-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - None",
 "Q9_OTHER-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - Other"]

ide_by_role = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
ide_by_role.dropna(inplace = True)
fig = px.treemap(ide_by_role , path = [px.Constant('Job Role') , 'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used IDE's based on the job role" , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above diagram is a breakup of IDE would be used based on the job role of a participant.

## Commonly used IDE's based on the participants age

In [ ]:
id_vars = ['Q1-What is your age (# years)?']
value_vars = ["Q9_Part_1-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - Jupyter (JupyterLab, Jupyter Notebooks, etc) ",
 "Q9_Part_2-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  RStudio ",
 "Q9_Part_3-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Visual Studio ",
 "Q9_Part_4-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Visual Studio Code (VSCode) ",
 "Q9_Part_5-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  PyCharm ",
 "Q9_Part_6-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Spyder  ",
 "Q9_Part_7-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Notepad++  ",
 "Q9_Part_8-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Sublime Text  ",
 "Q9_Part_9-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Vim / Emacs  ",
 "Q9_Part_10-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  MATLAB ",
 "Q9_Part_11-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Jupyter Notebook",
 "Q9_Part_12-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - None",
 "Q9_OTHER-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - Other"]

ide_by_age = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
ide_by_age.dropna(inplace = True)
fig = px.treemap(ide_by_age , path = [px.Constant('Age') , 'Q1-What is your age (# years)?' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used IDE's based on  the participants age" , 
                  height = 700 , width = 1000)

## Commonly used IDE's in each country

In [ ]:
id_vars = ['Q3-In which country do you currently reside?']
value_vars = ["Q9_Part_1-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - Jupyter (JupyterLab, Jupyter Notebooks, etc) ",
 "Q9_Part_2-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  RStudio ",
 "Q9_Part_3-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Visual Studio ",
 "Q9_Part_4-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Visual Studio Code (VSCode) ",
 "Q9_Part_5-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  PyCharm ",
 "Q9_Part_6-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Spyder  ",
 "Q9_Part_7-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Notepad++  ",
 "Q9_Part_8-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Sublime Text  ",
 "Q9_Part_9-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -   Vim / Emacs  ",
 "Q9_Part_10-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  MATLAB ",
 "Q9_Part_11-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Jupyter Notebook",
 "Q9_Part_12-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - None",
 "Q9_OTHER-Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - Other"]

ide_by_country = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
ide_by_country.dropna(inplace = True)
fig = px.treemap(ide_by_country , path = [px.Constant('Countries') , 
                                           'Q3-In which country do you currently reside?' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used IDE's in each country" , 
                  height = 700 , width = 1000)

# Commonly used Notebooks

In [ ]:
notebooks = [col for col in survey_copy if col.startswith('Q10')]
notebooks = survey_copy[notebooks]
notebooks.columns = ['Kaggle Notebooks' , 'Colab Notebooks' , 'Azure Notebooks' , 'Paperspace / Gradient' , 'Binder / JupyterHub' , 
               'Code Ocean' , 'IBM Watson Studio' , 'Amazon Sagemaker Studio Notebooks' , 'Amazon EMR Notebooks' , 
               'Google Cloud Notebooks' , 'Google Cloud Datalab' , 'Databricks Collaborative Notebooks' , 
               'Zeppelin / Zepl Notebooks' , 'Deepnote Notebooks' , 'Observable Notebooks' , 'None' , 'Other']
notebooks = (notebooks.count().to_frame().reset_index().rename(columns = {'index' : 'Notebooks' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
notebooks['Percentage'] = round(notebooks['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = notebooks['Count'] , y = notebooks['Notebooks'] , orientation = 'h' , text = notebooks['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most Commonly used Notebooks" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'Notebook')

<b>Observation</b>

- Majority of the most commonly used notebooks are Colab and Kaggle notebooks.
- It is surprising to see none in the top 3 of the commonly used notebooks. These might be some of the participants who do not know how to code as yet.

# Commonly used computing platform for data science projects

In [ ]:
comp_pltfrm = survey_copy['Q11-What type of computing platform do you use most often for your data science projects? - Selected Choice'].value_counts().reset_index()
comp_pltfrm.columns = ['Platform' , 'Count']
comp_pltfrm.sort_values(by = 'Count' , ascending = True , inplace = True)
comp_pltfrm['Percentage'] = round(comp_pltfrm['Count'] / comp_pltfrm['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = comp_pltfrm['Count'] , y = comp_pltfrm['Platform'] , orientation = 'h' , text = comp_pltfrm['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Commonly used computing platform for data science projects' , 
                 xaxis_title = 'Count' , yaxis_title = 'Computing Platform')

<b>Observation</b>

- Majority of the participants seem to use their personal laptop for their data science projects that they store on their local drive.
- Only 10% of the participants use an actual cloud platform for their projects.

# Commonly used Data Visualization tools or libraries

In [ ]:
vis_lib = [col for col in survey_copy if col.startswith('Q14')]
vis_lib = survey_copy[vis_lib]
vis_lib.columns = ['Matplotlib' , 'Seaborn' , 'Plotly / Plotly Express' , 'Ggplot / ggplot2' , 'Shiny' , 'D3 js' , 'Altair' , 
                   'Bokeh' , 'Geoplotlib' , 'Leaflet / Folium' , 'None' , 'Other']
vis_lib = (vis_lib.count().to_frame().reset_index().rename(columns = {'index' : 'Visual_Library' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
vis_lib['Percentage'] = round(vis_lib['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = vis_lib['Count'] , y = vis_lib['Visual_Library'] , orientation = 'h' , text = vis_lib['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most Commonly used visualization libraries" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'Visual_Library')

<b>Observation</b>

- Matplotlib is the most widely used visualization library, no doubt because this is the most basic visualization library there is, and everyone who starts off with data science starts visualizing using Matplotlib.
- Seaborn is closely followed by Matplotlib because it is built over it. Seaborn gives a slightly better visual effect than Matplotlib does.
- Plotly is an interactive visualization tool that is very helpful in interacting with data. It is slightly more complex than Matplotlib and Seaborn but it is worth the effort as it makes our notebooks visually appealing.

# Experience in using machine learning methods

In [ ]:
ml_methods = survey_copy['Q15-For how many years have you used machine learning methods?'].value_counts().reset_index()
ml_methods.columns = ['Experience' , 'Count']
ml_methods.sort_values(by = 'Count' , ascending = True , inplace = True)
ml_methods['Percentage'] = round(ml_methods['Count'] / ml_methods['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = ml_methods['Count'] , y = ml_methods['Experience'] , orientation = 'h' , text = ml_methods['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Experience in using machine learning methods' , 
                  xaxis_title = 'No. of Participants' , yaxis_title = 'ML Experience')

<b>Observation</b>

- Majority of the participants have less than two years of experience. This is known, as we seen most of the programmers who knew how to code only had 0-3 years of experience.
- We might be able to say that most of the participants learnt how to code with the idea of getting into the field of Data Science or Machine Learning.

## Machine Learning Experience based on Gender

In [ ]:
ml_exp_man = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Man']['Q15-For how many years have you used machine learning methods?'].value_counts()
ml_exp_woman = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Woman']['Q15-For how many years have you used machine learning methods?'].value_counts()
ml_exp_others = survey_copy[survey_copy['Q2-What is your gender? - Selected Choice'] == 'Others']['Q15-For how many years have you used machine learning methods?'].value_counts() 

## Calculating the percentage for each group
pct_man = [round((m / (m + w + o)) * 100, 1) for m , w , o in zip(ml_exp_man.values , ml_exp_woman.values , ml_exp_others.values)]
pct_woman = [round((w / (m + w + o)) * 100, 1) for m , w , o in zip(ml_exp_man.values , ml_exp_woman.values , ml_exp_others.values)]
pct_others = [round((o / (m + w + o)) * 100, 1) for m , w , o in zip(ml_exp_man.values , ml_exp_woman.values , ml_exp_others.values)]

fig = go.Figure(data = [
                go.Bar(name = 'Man' , x = ml_exp_man.index , y = ml_exp_man.values , text = pct_man , marker_color = colors[2]) , 
                go.Bar(name = 'Woman' , x = ml_exp_woman.index , y = ml_exp_woman.values , text = pct_woman , marker_color = colors[3]) , 
                go.Bar(name = 'Others' , x = ml_exp_others.index , y = ml_exp_others.values , text = pct_others , marker_color = colors[4])])

fig.update_traces(texttemplate='%{text:.3s}%', textposition='inside')

fig.update_layout(template = 'plotly_dark' , 
                  barmode = 'stack' , 
                  title = 'Machine Learning Experience based on Gender' , 
                  xaxis_title = 'Machine Learning Experience' , yaxis_title = 'Number of Participants')

## Machine learning experience based on gender and age

In [ ]:
sex_age_mlexp = survey_copy[survey_copy['Q15-For how many years have you used machine learning methods?'].notna()]


fig = px.treemap(sex_age_mlexp , path = [px.Constant('Gender') , 
                                                  'Q2-What is your gender? - Selected Choice' , 
                                                  'Q1-What is your age (# years)?' , 
                                                  'Q15-For how many years have you used machine learning methods?'])

fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Machine Learning experience distributed by gender and age" , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above is a treemap of machine learning experience based on the age group of the participants.

## Machine learning experience based on the job role

In [ ]:
job_mlexp = survey_copy[survey_copy['Q15-For how many years have you used machine learning methods?'].notna()]


fig = px.treemap(job_mlexp , path = [px.Constant('Job Role') , 
                                                  'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice' , 
                                                  'Q15-For how many years have you used machine learning methods?'])

fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Machine Learning experience based on the job role" , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above is a treemap of machine learning experience based on the job role of the participants.

# Most commonly used machine learning frameworks

In [ ]:
ml_framework = [col for col in survey_copy if col.startswith('Q16')]
ml_framework = survey_copy[ml_framework]
ml_framework.columns = ['Scikit-learn' , 'TensorFlow' , 'Keras' , 'PyTorch' , 'Fast.ai' , 'MXNet' , 'Xgboost' , 'LightGBM' , 
                        'CatBoost' , 'Prophet' , 'H2O 3' , 'Caret' , 'Tidymodels' , 'JAX' , 
                        'PyTorch Lightning' , 'Huggingface' , 'None' , 'Other']
ml_framework = (ml_framework.count().to_frame().reset_index().rename(columns = {'index' : 'ML_Framework' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
ml_framework['Percentage'] = round(ml_framework['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = ml_framework['Count'] , y = ml_framework['ML_Framework'] , orientation = 'h' , text = ml_framework['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most Commonly used machine learning frameworks" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'ML_Framework')

<b>Observation</b>

- As Data Scientists would know, some of the commonly used libraries in machine learning are Sckit-Learn , TensorFlow, Keras and PyTorch.
- Therefore, it is only fair that they are the most commonly used machine learning libraries/framworks.

# Most commonly used machine learning algorithms

In [ ]:
ml_algo = [col for col in survey_copy if col.startswith('Q17')]
ml_algo = survey_copy[ml_algo]
ml_algo.columns = ['Linear or Logistic Regression' , 'Decision Trees or Random Forests' , 'Gradient Boosting Machines' , 
                   'Bayesian Approaches' , 'Evolutionary Approaches' , 'Dense Neural Networks' , 'Convolutional Neural Networks' , 
                   'Generative Adversarial Networks' , 'Recurrent Neural Networks' , 'Transformer Networks' , 'None' , 'Other']
ml_algo = (ml_algo.count().to_frame().reset_index().rename(columns = {'index' : 'ML_Algorithm' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
ml_algo['Percentage'] = round(ml_algo['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = ml_algo['Count'] , y = ml_algo['ML_Algorithm'] , orientation = 'h' , text = ml_algo['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most Commonly used machine learning algorithms" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'ML_Algorithm')

<b>Observation</b>

- The most commonly and widely used algorithms are linear for regression and logistic for classification. These are the basic algorithms but sometimes proven to be efficient algorithms at times.
- Following these algorithms are tree based algorithms (Decision and Random Forest), followed by Gradient Boosting Methods.

# Most commonly used computer vision methods

In [ ]:
cv_methods = [col for col in survey_copy if col.startswith('Q18')]
cv_methods = survey_copy[cv_methods]
cv_methods.columns = ['General purpose image/video tools' , 'Image segmentation methods' , 'Object detection methods' , 
                      'Image classification and other general purpose networks' , 'Generative Networks' , 'None' , 'Other']
cv_methods = (cv_methods.count().to_frame().reset_index().rename(columns = {'index' : 'CV_Methods' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
cv_methods['Percentage'] = round(cv_methods['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = cv_methods['Count'] , y = cv_methods['CV_Methods'] , orientation = 'h' , text = cv_methods['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most Commonly used computer vision methods" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'CV_Methods')

# Most commonly used NLP methods

In [ ]:
nlp_methods = [col for col in survey_copy if col.startswith('Q19')]
nlp_methods = survey_copy[nlp_methods]
nlp_methods.columns = ['Word embeddings/vectors' , 'Encoder-decorder models' , 'Contextualized embeddings' , 
                      'Transformer language models' , 'None' , 'Other']
nlp_methods = (nlp_methods.count().to_frame().reset_index().rename(columns = {'index' : 'NLP_Methods' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
nlp_methods['Percentage'] = round(nlp_methods['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = nlp_methods['Count'] , y = nlp_methods['NLP_Methods'] , orientation = 'h' , text = nlp_methods['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most Commonly used NLP methods" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'NLP_Methods')

# Industry of current/recent employer

In [ ]:
industry = survey_copy['Q20-In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice'].value_counts().reset_index()
industry.columns = ['Industry' , 'Count']
industry.sort_values(by = 'Count' , ascending = True , inplace = True)
industry['Percentage'] = round(industry['Count'] / industry['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = industry['Count'] , y = industry['Industry'] , orientation = 'h' , text = industry['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Industry of current/recent employer' , 
                  xaxis_title = 'Count' , yaxis_title = 'Industry')

# Yearly compensation ($USD)

In [ ]:
compensation = survey_copy['Q25-What is your current yearly compensation (approximate $USD)?'].value_counts().reset_index()
compensation.columns = ['Compensation' , 'Count']
compensation.sort_values(by = 'Count' , ascending = True , inplace = True)
compensation['Percentage'] = round(compensation['Count'] / compensation['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(y = compensation['Compensation'] , x = compensation['Count'] ,  orientation = 'h' , text = compensation['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  autosize = False , 
                  width = 1000 , 
                  height = 1000 , 
                  title = 'Yearly compensation ($USD)' , 
                  xaxis_title = 'Count' , yaxis_title = 'Compensation')
fig.update_xaxes(tickangle = 90)

<b>Observation</b>

- Clearly we notice that majority of the income ranges from $0 to $999 which does not seem too fair to be honest depending on the education and job roles we have seen so far.
- One of the reason the income range is high is because most of the participants are fairly new to the field of Data Science and most of them were in the age group of 18-24. They might have just started out as interns in which they get paid a stipend for their work. However, this is just an assumption. There are many factors that play a role in deciding the salary of a person.

## Median yearly income by each country

In [ ]:
survey_copy[['min_w','max_w']] = survey_copy['Q25-What is your current yearly compensation (approximate $USD)?'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.replace('>', '', regex=False).str.split('-', expand = True)
survey_copy[['min_w','max_w']] = survey_copy[['min_w','max_w']].astype('float')
survey_copy['Mean_Compensation'] = (survey_copy['min_w'] + survey_copy['max_w'])/2 + 0.5
survey_copy['Mean_Compensation']

comp_by_country = survey_copy.groupby(['Q3-In which country do you currently reside?'])['Mean_Compensation'].median().sort_values(ascending = True)

fig = go.Figure(go.Bar(x = comp_by_country.values , y = comp_by_country.index , textposition = 'outside' , text = comp_by_country.values , orientation = 'h' , marker_color = colors[2]))

fig.update_traces(texttemplate = '%{text:.2s}')
fig.update_layout(template = 'plotly_dark' , 
                  autosize = False , 
                  width = 1000 , 
                  height = 1400 , 
                  title = 'Median yearly income by each country' , 
                  xaxis_title = 'Income' , yaxis_title = 'Country')

<b>Observation</b>

- Above is the median yearly income by each country.
- USA and Switzerland manage to top the list for highest median yearly incomes.

## Mean yearly income based on gender

In [ ]:
fig = px.box(data_frame = survey_copy , x = 'Mean_Compensation' , y = 'Q2-What is your gender? - Selected Choice')

fig.update_layout(template = 'plotly_dark' , 
                  title = 'Mean Compensation based on the gender' , 
                  xaxis_title = 'Mean Income' , yaxis_title = 'Gender')

## Mean yearly compensation based on the job profile

In [ ]:
fig = px.box(data_frame = survey_copy , x = 'Mean_Compensation' , y = 'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice')

fig.update_layout(template = 'plotly_dark' , 
                  title = 'Mean Compensation based on the job profile' , 
                  xaxis_title = 'Mean Income' , yaxis_title = 'Profile' , 
                  autosize = False , 
                  width = 1000 , 
                  height = 1000)

<b>Observation</b>

- Noticeably, Product Managers and Project Managers seem to have the highest mean yearly income, which could be true because of the position that they are in.
- The student and unemployed fields are naturally empty.
- Given the boom of Machine Learning right now, it is surprising to see that ML Engineers seem to have a very low average yearly income.

## Mean yearly compensation based on age

In [ ]:
fig = px.box(data_frame = survey_copy , x = 'Mean_Compensation' , y = 'Q1-What is your age (# years)?')

fig.update_layout(template = 'plotly_dark' , 
                  title = 'Mean Compensation based on age' , 
                  xaxis_title = 'Mean Income' , yaxis_title = 'Age' , 
                  autosize = False , 
                  width = 1000 , 
                  height = 1000)

<b>Observation</b>

- We all know that as age increases, experience increases and we can visually see that as experience increases, the mean salary as well tends to increase.

## Approximate amount spent on machine learning in the last 5 years ($USD)

In [ ]:
ml_amt_spent = survey_copy['Q26-Approximately how much money have you (or your team) spent on machine learning and/or cloud computing services at home (or at work) in the past 5 years (approximate $USD)?'].value_counts().reset_index()
ml_amt_spent.columns = ['Amount Spent' , 'Count']
ml_amt_spent.sort_values(by = 'Count' , ascending = True , inplace = True)
ml_amt_spent['Percentage'] = round(ml_amt_spent['Count'] / ml_amt_spent['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(y = ml_amt_spent['Amount Spent'] , x = ml_amt_spent['Count'] ,  orientation = 'h' , text = ml_amt_spent['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  autosize = False , 
                  width = 1000 , 
                  height = 600 , 
                  title = 'Amount spent on machine learning in the last 5 years ($USD)' , 
                  xaxis_title = 'Count' , yaxis_title = 'Amount Spent($USD)' , 
                  yaxis = dict(tickmode = 'array' , tickvals = [5,4,3,2,1,0] , ticktext = ['0','100-999','1,000-9,999','1-99','10,000-99,999','> = 100,000']))

# Cloud computing platforms used on a regular basis

In [ ]:
cld_compute = [col for col in survey_copy if col.startswith('Q27_A')]
cld_compute = survey_copy[cld_compute]
cld_compute.columns = ['Amazon Web Services' , 'Microsoft Azure' , 'Google Cloud Platform' , 'IBM Cloud / Red Hat' , 
                       'Oracle Cloud' , 'SAP Cloud' , 'Salesforce Cloud' , 'VMware Cloud' , 'Alibaba Cloud' , 
                       'Tencent Cloud' , 'None' , 'Other']
cld_compute = (cld_compute.count().to_frame().reset_index().rename(columns = {'index' : 'Cloud Platform' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
cld_compute['Percentage'] = round(cld_compute['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = cld_compute['Count'] , y = cld_compute['Cloud Platform'] , orientation = 'h' , text = cld_compute['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most Commonly used Cloud Platforms" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'Cloud Platform')

<b>Observation</b>

- As we all know, AWS,GCP and MS-Azure are the 3 top cloud computing platforms that individuals/organizations use in today's world.
- Therefore it is not a surprise that AWS, GCP and Azure are among the top 4 of the platforms that are being used.
- 'None' is a part of top three. This maybe due to various reasons as it is an individual's choice whether or not to use the cloud for their projects. As we have seen before some of the participants tend to store their work done mainly on their personal Laptops/Desktops rather than the cloud.

## Commonly used cloud platforms based on the job role

In [ ]:
id_vars = ['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice']
value_vars = ['Q27_A_Part_1-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  Amazon Web Services (AWS) ',
 'Q27_A_Part_2-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  Microsoft Azure ',
 'Q27_A_Part_3-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  Google Cloud Platform (GCP) ',
 'Q27_A_Part_4-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  IBM Cloud / Red Hat ',
 'Q27_A_Part_5-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  Oracle Cloud ',
 'Q27_A_Part_6-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  SAP Cloud ',
 'Q27_A_Part_7-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  Salesforce Cloud ',
 'Q27_A_Part_8-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  VMware Cloud ',
 'Q27_A_Part_9-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  Alibaba Cloud ',
 'Q27_A_Part_10-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice -  Tencent Cloud ',
 'Q27_A_Part_11-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice - None',
 'Q27_A_OTHER-Which of the following cloud computing platforms do you use on a regular basis? (Select all that apply) - Selected Choice - Other']

cld_plt_by_role = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
cld_plt_by_role.dropna(inplace = True)
fig = px.treemap(cld_plt_by_role , path = [px.Constant('Job Role') , 'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used cloud platforms based on the job role" , 
                  height = 700 , width = 1000)

<b>Observation</b>

- Above is a treemap of commonly used cloud platforms based on the job role of participants.
- Most of the people from different job roles tend to use some sort of cloud platforms.
- We notice that for certains roles such as Data Analyst, Research Scientist and Statistician, most of the prople do not use any cloud platforms and it maybe for various reasons.

# Most enjoyable cloud platform to use

In [ ]:
best_cld_pltfm = survey_copy['Q28-Of the cloud platforms that you are familiar with, which has the best developer experience (most enjoyable to use)? - Selected Choice'].value_counts().reset_index()
best_cld_pltfm.columns = ['Cloud Platform' , 'Count']
best_cld_pltfm.sort_values(by = 'Count' , ascending = True , inplace = True)
best_cld_pltfm['Percentage'] = round(best_cld_pltfm['Count'] / best_cld_pltfm['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = best_cld_pltfm['Count'] , y = best_cld_pltfm['Cloud Platform'] , orientation = 'h' , text = best_cld_pltfm['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Most enjoyable cloud platform to use' , 
                  xaxis_title = 'Count' , yaxis_title = 'Cloud Platform')

# Most widely used Machine Learning Cloud Products

In [ ]:
cld_ml_prod = [col for col in survey_copy if col.startswith('Q31_A')]
cld_ml_prod = survey_copy[cld_ml_prod]
cld_ml_prod.columns = ['Amazon SageMaker' , 'Azure Machine Learning Studio' , 'Google Cloud Vertex AI' , 'DataRobot' , 
                       'Databricks' , 'Dataiku' , 'Alteryx' , 'Rapidminer' , 'None' , 'Other']
cld_ml_prod = (cld_ml_prod.count().to_frame().reset_index().rename(columns = {'index' : 'ML Cloud Products' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
cld_ml_prod['Percentage'] = round(cld_ml_prod['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = cld_ml_prod['Count'] , y = cld_ml_prod['ML Cloud Products'] , orientation = 'h' , text = cld_ml_prod['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most Commonly used Machine Learning Cloud Products" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'ML Cloud Products')

# Regularly used Big Data Products

In [ ]:
big_data_prod = [col for col in survey_copy if col.startswith('Q32_A')]
big_data_prod = survey_copy[big_data_prod]
big_data_prod.columns = ['MySQL' , 'PostgreSQL' , 'SQLite' , 'Oracle Database' , 'MongoDB' , 'Snowflake' , 'IBM Db2' , 
                         'Microsoft SQL Server' , 'Microsoft Azure SQL Database' , 'Microsoft Azure Cosmos DB' , 
                         'Amazon Redshift' , 'Amazon Aurora' , 'Amazon RDS' , 'Amazon DynamoDB' , 
                         'Google Cloud BigQuery' , 'Google Cloud SQL' , 'Google Cloud Firestore' , 'Google Cloud BigTable' , 
                         'Google Cloud Spanner' , 'None' , 'Other']
big_data_prod = (big_data_prod.count().to_frame().reset_index().rename(columns = {'index' : 'Big Data Products' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
big_data_prod['Percentage'] = round(big_data_prod['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = big_data_prod['Count'] , y = big_data_prod['Big Data Products'] , orientation = 'h' , text = big_data_prod['Percentage'] , textposition = 'outside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Regularly used Big Data Products" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'Big Data Products')

<b>Observation</b>

- As we can see MySQL, PostgreSQL, MS-SQL Server and MongoDB are some of the most commonly used Databases in today's world.
- Surprisingly None is part of the top three. Looks like some people do not use databases to store/manage their data.

## Commonly used big data products based on the job role

In [ ]:
id_vars = ['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice']
value_vars = ['Q32_A_Part_1-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - MySQL ',
 'Q32_A_Part_2-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - PostgreSQL ',
 'Q32_A_Part_3-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - SQLite ',
 'Q32_A_Part_4-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Oracle Database ',
 'Q32_A_Part_5-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - MongoDB ',
 'Q32_A_Part_6-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Snowflake ',
 'Q32_A_Part_7-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - IBM Db2 ',
 'Q32_A_Part_8-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Microsoft SQL Server ',
 'Q32_A_Part_9-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Microsoft Azure SQL Database ',
 'Q32_A_Part_10-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Microsoft Azure Cosmos DB ',
 'Q32_A_Part_11-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Amazon Redshift ',
 'Q32_A_Part_12-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Amazon Aurora ',
 'Q32_A_Part_13-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Amazon RDS ',
 'Q32_A_Part_14-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Amazon DynamoDB ',
 'Q32_A_Part_15-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Google Cloud BigQuery ',
 'Q32_A_Part_16-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Google Cloud SQL ',
 'Q32_A_Part_17-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Google Cloud Firestore ',
 'Q32_A_Part_18-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Google Cloud BigTable ',
 'Q32_A_Part_19-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Google Cloud Spanner ',
 'Q32_A_Part_20-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - None',
 'Q32_A_OTHER-Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you use on a regular basis? (Select all that apply) - Selected Choice - Other']

big_data_prd_by_role = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
big_data_prd_by_role.dropna(inplace = True)
fig = px.treemap(big_data_prd_by_role , path = [px.Constant('Job Role') , 'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used big data products based on the job role" , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above treemap is a diagramatic representation of what database people use most based on their job role.

# Most often used Big Data Products

In [ ]:
best_cld_pltfm = survey_copy['Q33-Which of the following big data products (relational database, data warehouse, data lake, or similar) do you use most often? - Selected Choice'].value_counts().reset_index()
best_cld_pltfm.columns = ['Big Data Products' , 'Count']
best_cld_pltfm.sort_values(by = 'Count' , ascending = True , inplace = True)
best_cld_pltfm['Percentage'] = round(best_cld_pltfm['Count'] / best_cld_pltfm['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = best_cld_pltfm['Count'] , y = best_cld_pltfm['Big Data Products'] , orientation = 'h' , text = best_cld_pltfm['Percentage'] , textposition = 'outside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Most often used Big Data Products' , 
                  xaxis_title = 'Count' , yaxis_title = 'Big Data Products')

# Most regularly used Business Intelligence(BI) Tools

In [ ]:
bi_tools_reg = [col for col in survey_copy if col.startswith('Q34_A')]
bi_tools_reg = survey_copy[bi_tools_reg]
bi_tools_reg.columns = ['Amazon QuickSight' , 'Microsoft Power BI' , 'Google Data Studio' , 'Looker' , 'Tableau' , 'Salesforce' , 
                        'Tableau CRM' , 'Qlik' , 'Domo' , 'TIBCO Spotfir' , 'Alteryx' , 'Sisense' , 'SAP Analytics Cloud' , 
                        'Microsoft Azure Synapse' , 'Thoughtspot' , 'None' , 'Other']
bi_tools_reg = (bi_tools_reg.count().to_frame().reset_index().rename(columns = {'index' : 'BI Tools' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
bi_tools_reg['Percentage'] = round(bi_tools_reg['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = bi_tools_reg['Count'] , y = bi_tools_reg['BI Tools'] , orientation = 'h' , text = bi_tools_reg['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Regularly used BI Tools" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'BI Tools')

## Commonly used BI Tool based on the job profile

In [ ]:
id_vars = ['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice']
value_vars = ['Q34_A_Part_1-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Amazon QuickSight',
 'Q34_A_Part_2-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Microsoft Power BI',
 'Q34_A_Part_3-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Google Data Studio',
 'Q34_A_Part_4-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Looker',
 'Q34_A_Part_5-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Tableau',
 'Q34_A_Part_6-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Salesforce',
 'Q34_A_Part_7-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Tableau CRM',
 'Q34_A_Part_8-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Qlik',
 'Q34_A_Part_9-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Domo',
 'Q34_A_Part_10-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - TIBCO Spotfire',
 'Q34_A_Part_11-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Alteryx ',
 'Q34_A_Part_12-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Sisense ',
 'Q34_A_Part_13-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - SAP Analytics Cloud ',
 'Q34_A_Part_14-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Microsoft Azure Synapse ',
 'Q34_A_Part_15-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Thoughtspot ',
 'Q34_A_Part_16-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - None',
 'Q34_A_OTHER-Which of the following business intelligence tools do you use on a regular basis? (Select all that apply) - Selected Choice - Other']

bi_tools_by_role = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
bi_tools_by_role.dropna(inplace = True)
fig = px.treemap(bi_tools_by_role , path = [px.Constant('Job Role') , 'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used BI Tools based on the job role" , 
                  height = 700 , width = 1000)

<b>Observation</b>

- Majority of the job roles do not use any BI tools for their data visualization.
- The only roles that use some sort of visualization are analyst roles such as Data Analyst, Business Analyst, Data Engineer and Product Manager.
- For everyone else, majority of them do not use these BI tools.

# Most often used BI Tools

In [ ]:
best_bi_tools = survey_copy['Q35-Which of the following business intelligence tools do you use most often? - Selected Choice'].value_counts().reset_index()
best_bi_tools.columns = ['BI Tools' , 'Count']
best_bi_tools.sort_values(by = 'Count' , ascending = True , inplace = True)
best_bi_tools['Percentage'] = round(best_bi_tools['Count'] / best_cld_pltfm['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = best_bi_tools['Count'] , y = best_bi_tools['BI Tools'] , orientation = 'h' , text = best_bi_tools['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Most often used BI Tools' , 
                  xaxis_title = 'Count' , yaxis_title = 'BI Tools')

# Often used Automated Machine Learning Tools

In [ ]:
auto_ml_tools = [col for col in survey_copy if col.startswith('Q37_A')]
auto_ml_tools = survey_copy[auto_ml_tools]
auto_ml_tools.columns = ['Google Cloud AutoML' , 'H2O Driverless AI' , 'Databricks AutoML' , 'DataRobot AutoML' , 
                         'Amazon Sagemaker Autopilot' , 'Azure Automated Machine Learning' , 'None' , 'Other']
auto_ml_tools = (auto_ml_tools.count().to_frame().reset_index().rename(columns = {'index' : 'AutoML Tools' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
auto_ml_tools['Percentage'] = round(auto_ml_tools['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = auto_ml_tools['Count'] , y = auto_ml_tools['AutoML Tools'] , orientation = 'h' , text = auto_ml_tools['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Often used AutoML Tools" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'AutoML Tools')

# Commonly used platform for sharing data analysis / machine learning applications

In [ ]:
sharing_platform = [col for col in survey_copy if col.startswith('Q39')]
sharing_platform = survey_copy[sharing_platform]
sharing_platform.columns = ['Plotly Dash' , 'Streamlit' , 'NBViewer' , 'GitHub' , 'Personal blog' , 
                            'Kaggle' , 'Colab' , 'Shiny' , 'None' , 'Other']
sharing_platform = (sharing_platform.count().to_frame().reset_index().rename(columns = {'index' : 'Platform' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
sharing_platform['Percentage'] = round(sharing_platform['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = sharing_platform['Count'] , y = sharing_platform['Platform'] , orientation = 'h' , text = sharing_platform['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Platform for sharing data analysis / machine learning applications" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'Platform')

<b>Observation</b>

- Seeing how famous GitHub is, it is not surprising that it tops the list as the most famous platform to share your data analysis or machine learning applications.
- Some of the people prefer not to share their work at all on any of these platforms, which is a bit surprising seeing that one might get recognized only upon others seeing their work.
- Kaggle is another major platform for sharing our projects.

# Most widely used platform to study Data Science

In [ ]:
ds_platform = [col for col in survey_copy if col.startswith('Q40')]
ds_platform = survey_copy[ds_platform]
ds_platform.columns = ['Coursera' , 'edX' , 'Kaggle Learn Courses' , 'DataCamp' , 'Fast.ai' , 'Udacity' , 'Udemy' , 
                       'LinkedIn Learning' , 'Cloud-certification programs' , 'University Courses' , 'None' , 'Other']
ds_platform = (ds_platform.count().to_frame().reset_index().rename(columns = {'index' : 'Data Science Learning Platform' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
ds_platform['Percentage'] = round(ds_platform['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = ds_platform['Count'] , y = ds_platform['Data Science Learning Platform'] , orientation = 'h' , text = ds_platform['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Most widely used platform to study Data Science" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'Data Science Learning Platform')

<b>Observation</b>

- Given the situation we are in today(Covid-19), it is not surprising to see that many people tend to learn Data Science from an online platform. 
- In this case, Coursera tops the list, closely followed by Kaggle and then Udemy.

## Commonly used platforms to study Data Science based on the job profile

In [ ]:
id_vars = ['Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice']
value_vars = ['Q40_Part_1-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Coursera',
 'Q40_Part_2-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - edX',
 'Q40_Part_3-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Kaggle Learn Courses',
 'Q40_Part_4-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - DataCamp',
 'Q40_Part_5-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Fast.ai',
 'Q40_Part_6-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Udacity',
 'Q40_Part_7-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Udemy',
 'Q40_Part_8-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - LinkedIn Learning',
 'Q40_Part_9-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Cloud-certification programs (direct from AWS, Azure, GCP, or similar)',
 'Q40_Part_10-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - University Courses (resulting in a university degree)',
 'Q40_Part_11-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - None',
 'Q40_OTHER-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Other']

course_pltfrm_by_role = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
course_pltfrm_by_role.dropna(inplace = True)
fig = px.treemap(course_pltfrm_by_role , path = [px.Constant('Job Role') , 'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used platforms to study Data Science based on the job profile" , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above diagram is a breakup of what platform people use most to study based on their job profile.

## Commonly used platforms to study Data Science based on age

In [ ]:
id_vars = ['Q1-What is your age (# years)?']
value_vars = ['Q40_Part_1-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Coursera',
 'Q40_Part_2-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - edX',
 'Q40_Part_3-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Kaggle Learn Courses',
 'Q40_Part_4-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - DataCamp',
 'Q40_Part_5-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Fast.ai',
 'Q40_Part_6-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Udacity',
 'Q40_Part_7-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Udemy',
 'Q40_Part_8-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - LinkedIn Learning',
 'Q40_Part_9-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Cloud-certification programs (direct from AWS, Azure, GCP, or similar)',
 'Q40_Part_10-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - University Courses (resulting in a university degree)',
 'Q40_Part_11-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - None',
 'Q40_OTHER-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Other']

course_pltfrm_by_age = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
course_pltfrm_by_age.dropna(inplace = True)
fig = px.treemap(course_pltfrm_by_age , path = [px.Constant('Age') , 'Q1-What is your age (# years)?' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used platforms to study Data Science based on age" , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above diagram is a breakup of what platform people use most to study based on their age group.

## Most commonly used platforms to study Data Science by Country

In [ ]:
id_vars = ['Q3-In which country do you currently reside?']
value_vars = ['Q40_Part_1-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Coursera',
 'Q40_Part_2-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - edX',
 'Q40_Part_3-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Kaggle Learn Courses',
 'Q40_Part_4-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - DataCamp',
 'Q40_Part_5-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Fast.ai',
 'Q40_Part_6-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Udacity',
 'Q40_Part_7-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Udemy',
 'Q40_Part_8-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - LinkedIn Learning',
 'Q40_Part_9-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Cloud-certification programs (direct from AWS, Azure, GCP, or similar)',
 'Q40_Part_10-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - University Courses (resulting in a university degree)',
 'Q40_Part_11-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - None',
 'Q40_OTHER-On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Other']

course_pltfrm_by_ctry = pd.melt(survey_copy , id_vars = id_vars , value_vars = value_vars , var_name = 'Question' , value_name = 'Answer')
course_pltfrm_by_ctry.dropna(inplace = True)
fig = px.treemap(course_pltfrm_by_ctry , path = [px.Constant('Countries') , 'Q3-In which country do you currently reside?' , 'Answer'])
fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "Commonly used platforms to study Data Science by country" , 
                  height = 700 , width = 1000)

<b>Observation</b>

- The above diagram is a breakup of what platform people use most to study based on the country they are from.

# Most often used tools to analyze data

In [ ]:
data_tools = survey_copy['Q41-What is the primary tool that you use at work or school to analyze data? (Include text response) - Selected Choice'].value_counts().reset_index()
data_tools.columns = ['Tools to analyze data' , 'Count']
data_tools.sort_values(by = 'Count' , ascending = True , inplace = True)
data_tools['Percentage'] = round(data_tools['Count'] / data_tools['Count'].sum() * 100 , 2)

fig = go.Figure(go.Bar(x = data_tools['Count'] , y = data_tools['Tools to analyze data'] , orientation = 'h' , text = data_tools['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = 'Most often used tools to analyze data' , 
                  xaxis_title = 'Count' , yaxis_title = 'Tools to analyze data' , 
                  autosize = False , 
                  width = 1000)

## Most often used tools to analyze data based on the job role

In [ ]:
tools = survey_copy[survey_copy['Q41-What is the primary tool that you use at work or school to analyze data? (Include text response) - Selected Choice'].notna()]


fig = px.treemap(tools , path = [px.Constant('Job Role') , 
                                 'Q5-Select the title most similar to your current role (or most recent title if retired): - Selected Choice' , 
                                 'Q41-What is the primary tool that you use at work or school to analyze data? (Include text response) - Selected Choice'])

fig.update_traces(textinfo = 'label+percent parent')
fig.update_layout(template = 'plotly_dark' , 
                  title = "MMost often used tools to analyze data based on the job role" , 
                  height = 700 , width = 1000)

# Favorite Media Sources for Data Science Topics

In [ ]:
media_sources = [col for col in survey_copy if col.startswith('Q42')]
media_sources = survey_copy[media_sources]
media_sources.columns = ['Twitter' , 'Email newsletters' , 'Reddit' , 'Kaggle' , 'Course Forums' , 'YouTube' , 'Podcasts' , 
                       'Blogs' , 'Journal Publications' , 'Slack Communities' , 'None' , 'Other']
media_sources = (media_sources.count().to_frame().reset_index().rename(columns = {'index' : 'Media Sources' , 0 : 'Count'}).sort_values(by = ['Count'] , ascending = True))
media_sources['Percentage'] = round(media_sources['Count'] / len(survey_copy) * 100 , 2)

fig = go.Figure(go.Bar(x = media_sources['Count'] , y = media_sources['Media Sources'] , orientation = 'h' , text = media_sources['Percentage'] , textposition = 'inside' , texttemplate='%{text:.3s}%' , marker_color = colors[2]))
fig.update_layout(template = 'plotly_dark' , 
                  title = "Favorite Media Sources for Data Science Topics" , 
                  xaxis_title = 'Number of Participants' , yaxis_title = 'Media Sources')

<b>Observation</b>

- The most commonly used media sources for Data Science topics are Kaggle, YouTube and Personal Blogs.

# <center>Takeaways</center>

## <center>The Typical Kaggle Data Scientist in 2021</center>

- <b>AGE : </b>18-29 years of age.
- <b>GENDER : </b>Male.
- <b>COUNTRY : </b>Either lives in India or USA.
- <b>EDUCATION : </b>Has either completed his/her Bachelor's or Master's Degree.
- <b>JOB ROLE : </b>Currently studying.
- <b>WORKING INDUSTRY : </b>Technology/Academics.
- <b>CODING EXPERIENCE : </b>Less than 3 years of coding experience.
- <b>PROGRAMMING LANGUAGES : </b>Mostly uses either Python or SQL.
- <b>INTEGRATED DEVELOPMENT ENVIRONMENT(IDE) : </b>Mostly works with Jupyter Notebook or VS Code.
- <b>NOTEBOOKS : </b>Mostly uses either Colab or Kaggle Notebooks.
- <b>COMPUTING PLATFORM : </b>Laptop.
- <b>MACHINE LEARNING EXPERIENCE : </b>Less than a year.
- <b>MACHINE LEARNING FRAMEWORKS : </b>Scikit-Learn and TensorFlow.
- <b>MACHINE LEARNING ALGORITHMS : </b>Linear Regression/Logistic Regression/Decision Trees/Random Forest algorithms.
- <b>DATA VISUALIZATION TOOLS : </b>Matplotlib/Seaborn/Plotly.
- <b>YEARLY COMPENSATION : </b>Less than $1000.
- <b>CLOUD COMPUTING PLATFORMS : </b>Uses AWS or GCP most often.
- <b>BIG DATA PLATFORMS : </b>MySQL or PostgreSQL.
- <b>BUSINESS INTELLIGENCE TOOLS : </b>Microsoft PowerBI or Tableau.
- <b>SHARING PLATFORMS : </b>GitHub and Kaggle.
- <b>PLATFORMS TO STUDY DATA SCIENCE : </b>Coursera and Kaggle.
- <b>FAVOURITE MEDIA SOURCES : </b>Kaggle and YouTube.

# <center>Conclusion</center>

- I just want to thank Kaggle for the opportunity to get to work on this dataset. I myself am still a beginner in the field of Data Science and Machine Learning and I know that there is a long way to go. I had a lot of fun exploring different ways to tackle problems this datset had to offer. I did my best to bring out as many insights from the data that I could. Please feel free to ask  me personally if you have any questions and I would be glad to answer them. Looking forward to many more competitions like this in the future.\

- <b>I hope that Kagglers who read this notebook enjoyed it and found something useful out of it. If you do like the work that I did, I'd appreciate it if you let me know in the comments. I wish you all the best in your Data Science journey. Thanks for reading once again and Good Luck :)</b>